In [1]:
import pandas as pd
import json



df = pd.read_csv("JoSAA 2022 Seat Allocation Tool - For Surya.csv", header=0)
# print(df.head())
seat_types = df["Seat Type"].unique()
print(seat_types)

['OPEN' 'EWS' 'OBC-NCL' 'SC' 'ST' 'OPEN (PwD)' 'EWS (PwD)' 'OBC-NCL (PwD)'
 'SC (PwD)' 'ST (PwD)']


In [ ]:
for seat_type in seat_types:
    df_seat_type = df[df["Seat Type"] == seat_type]
    df_seat_type.to_json(f"public/data/{seat_type}.json", orient="records")

In [42]:
df.columns

Index(['College Rank', 'State', 'Exam', 'Institute', 'Academic Program Name',
       'Quota', 'Special Quota', 'Seat Type', 'Gender', 'Round',
       'Opening Rank', 'Closing Rank', 'Input Rank Comparison to Closing Rank',
       'Category Match', '500', 'OBC-NCL'],
      dtype='object')

In [18]:
df["Gender"].unique()

array(['Gender-Neutral', 'Female-only (including Supernumerary)'],
      dtype=object)

In [22]:
df["Exam"].unique()

array(['JEE Advanced', 'JEE Main'], dtype=object)

In [23]:
df["State"].unique()

array(['Odisha', 'Maharashtra', 'Himachal Pradesh', 'New Delhi',
       'Madhya Pradesh', 'West Bengal', 'Telangana', 'Rajasthan',
       'Uttar Pradesh', 'Tamil Nadu', 'Gujarat', 'Bihar', 'Uttarakhand',
       'Jharkhand', 'Punjab', 'Assam', 'Chhatisgarh', 'Goa', 'Kerala',
       'Andhra Pradesh', 'Jammu & Kashmir', 'Karnataka', 'Tripura',
       'Meghalaya', 'Nagaland', 'Puducherry', 'Sikkim',
       'Arunachal Pradesh', 'Haryana', 'Manipur', 'Mizoram', 'Diu'],
      dtype=object)

In [19]:
df["Round"].max()

6

In [28]:
df["Quota"].unique()

array(['AI', 'HS', 'OS'], dtype=object)

In [ ]:
# re rank based on round 1 ke open category ka closing ranks!

In [27]:
df[df["Institute"] == "National Institute of Technology Karnataka, Surathkal"]["Quota"].unique()

array(['HS', 'OS'], dtype=object)

In [32]:
neet_report = pd.read_csv("Admitted Candidates List Upto Mop Up Round MBBS BD.xlsx - Report.csv")
neet_stud = pd.read_csv("Admitted Candidates List Upto Mop Up Round MBBS BD.xlsx - Table 1.csv", header=1)

neet_stud.columns, neet_report.columns

(Index(['S. No.', 'Rollno', 'Name', 'Quota', 'AIR', 'Category', 'optionN o',
        'Institute Code', 'Institute Name', 'Subject', 'Allotted Category',
        'Allotte d ph', 'Admitte dRound'],
       dtype='object'),
 Index(['Institute Name', 'State', 'Subject', 'General', 'EW', 'SC', 'ST',
        'OBC', 'General_PWD', 'EW_PWD', 'SC_PWD', 'ST_PWD', 'OBC_PWD',
        'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
        'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
        'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24'],
       dtype='object'))

In [186]:
from collections import defaultdict

institute_code_to_names = defaultdict(list)
institute_name_to_codes = {}
for it, row in neet_stud.iterrows():
    code, name = row["Institute Code"], row["Institute Name"]

    if name not in institute_code_to_names[code]:
        institute_code_to_names[code].append(name)

    institute_name_to_codes[name] = code

In [ ]:
for k, v in institute_code_to_names.items():
    if len(v) > 1:
        print(k, v)
        break

In [167]:
# i am picking the first one that appears
code_to_college_mapper = {}
for k, v in institute_code_to_names.items():
    code_to_college_mapper[k] = v[0]

assert len(institute_code_to_names) == len(code_to_college_mapper)

In [168]:
# for each round, for each subject, for each quota, 
# using ph as pwd -- physically handicapped
# consider allotted cat or actual cat?
# what to do about quota? Employee State Insurance, etc.
# all of this should be for each institute na ... institute code!!
deter_cols = ["Quota", "Admitte dRound", "Allotte d ph", "Subject"]

output_cols = ["Institute Code", "Institute Name"]

close_mapper = defaultdict(lambda : 0)
open_mapper = defaultdict(lambda : float("inf"))

for it, row in neet_stud.iterrows():

    deter_col_vals = [row[x] for x in deter_cols]
    
    cat = row["Allotted Category"]
    if cat == "OP":
        cat = "General"
    
    deter_col_vals.append(cat)

    inst_code = row["Institute Code"]
    output_col_vals = [inst_code, code_to_college_mapper[inst_code]]

    all_cols = deter_col_vals + output_col_vals

    all_cols = tuple(all_cols) # hashable

    if close_mapper[all_cols] < row["AIR"]:
        close_mapper[all_cols] = row["AIR"]

    if open_mapper[all_cols] > row["AIR"]:
        open_mapper[all_cols] = row["AIR"]




In [169]:
for k, v in close_mapper.items():
    print(k, v)
    break

('Open Seat Quota', 1, 'NO', 'MBBS', 'General', 200502, 'AIIMS, New Delhi,AIIMS ANSARI NAGAR EAST AUROBINDO MARG NEW DELHI 110029, Delhi (NCT), 110029') 55.0


In [170]:
cat_unique_values = ["SC", "ST", "General", "OBC", "EW"]

In [171]:
all_dic = []
selectors = [0, 1, 2, 3, 4, 5, 6]
selector_names = ["Quota", "Round", "is_PWD", "Subject", "Seat Type", "Institute Code", "Institute Name"]

for k, v in close_mapper.items():
    dic = {}
    for key_index, name in zip(selectors, selector_names):
        dic[name] = k[key_index]

    if k in open_mapper:
        dic["Opening Rank"] = open_mapper[k]

    dic["Closing_Rank"] = v
    
    all_dic.append(dic)

    # break

all_dic = pd.DataFrame(all_dic)

In [172]:
# time to rank institutes!!
# find round 1 ka opening rank.. rank based on it!

code_to_open_ranks = defaultdict(lambda : float("inf"))

for it, row in all_dic.iterrows():
    # print(row["Round"] == 1 , row["Category"] == "General" , row["is_PWD"] == "NO", row["Quota"] == "Open Seat Quota")
    if code_to_open_ranks[row["Institute Code"]] > row["Opening Rank"]:
        code_to_open_ranks[row["Institute Code"]] = row["Opening Rank"]

        # break

In [173]:
# now sort the dict based on this
# sort keys of dict based on values
ranking = sorted(code_to_open_ranks.items(), key = lambda item: item[1])

code_to_rank_mapper = {}

rank_idx = 1
for k, v in ranking:
    code_to_rank_mapper[k] = rank_idx
    rank_idx += 1

In [174]:
len(all_dic["Institute Code"].unique()), len(code_to_open_ranks)

(483, 483)

In [175]:
all_dic["College Rank"] = all_dic["Institute Code"].map(code_to_rank_mapper)

In [177]:
# combine pwd and category
all_dic["Seat Type"] = all_dic.apply(lambda row: row["Seat Type"] + " (PwD)" if row["is_PWD"] != "NO" else row["Seat Type"], axis = 1)

In [151]:
all_dic.columns, df.columns

(Index(['Quota', 'Round', 'is_PWD', 'Subject', 'Category', 'Institute Code',
        'Institute Name', 'Opening Rank', 'Closing_Rank', 'College Rank'],
       dtype='object'),
 Index(['College Rank', 'State', 'Exam', 'Institute', 'Academic Program Name',
        'Quota', 'Special Quota', 'Seat Type', 'Gender', 'Round',
        'Opening Rank', 'Closing Rank', 'Input Rank Comparison to Closing Rank',
        'Category Match', '500', 'OBC-NCL'],
       dtype='object'))

In [183]:
all_dic = all_dic.rename(columns={
    "Category": "Seat Type",
    "Institute Name": "Institute",
    "Subject": "Academic Program Name",
    "Closing_Rank": "Closing Rank"
}) 

In [179]:
def replace_cat(row):
    cat = row["Seat Type"]
    if cat == "General":
        cat = "OPEN"
    elif cat == "General (PwD)":
        cat = "OPEN (PwD)"
    elif cat == "EW":
        cat = "EWS"
    elif cat == "EW (PwD)":
        cat = "EWS (PwD)"
    elif cat == "OBC":
        cat = "OBC-NCL"
    elif cat == "OBC (PwD)":
        cat = "OBC-NCL (PwD)"

    return cat

all_dic["Seat Type"] = all_dic.apply(lambda row: replace_cat(row), axis = 1)



In [187]:
# figuring out state for each..
# merge on institute name
print(neet_stud.shape, neet_report.shape)
inst_code_to_state_map = {}
for it, row in pd.merge(neet_stud, neet_report, on=["Institute Name"]).iterrows():
    name, state = row["Institute Name"], row["State"]

    inst_code_to_state_map[institute_name_to_codes[name]] = state



all_dic["State"] = all_dic["Institute Code"].map(inst_code_to_state_map)


(23059, 13) (433, 25)


In [188]:
for cat in all_dic["Seat Type"].unique():
    all_dic_category = all_dic[all_dic["Seat Type"] == cat]

    all_dic_category.to_json(f"public/data/NEET/{cat}.json", orient="records")

In [190]:
all_dic.to_csv("Surya - NEET Data.csv")

In [ ]:
schol_data = pd.read_csv("scholarship_data.csv")

In [4]:
# update state names
names_in_sheet = []
for it, row in schol_data.iterrows():
    for k, v in row.items():
        if "State" in k:
            if v not in names_in_sheet:
                names_in_sheet.append(v)

            

In [5]:
names_in_sheet

[nan,
 'Andhra Pradesh',
 'Karnataka',
 'Telangana',
 'Gujarat',
 'Maharashtra',
 'Tamil Nadu',
 'Bihar',
 'Haryana',
 'Kerala',
 'Arunachal Pradesh',
 'Assam',
 'Manipur',
 'Meghalaya',
 'Mizoram',
 'Nagaland',
 'Sikkim',
 'Tripura']

In [6]:
# update state names
city_names_in_sheet = []
for it, row in schol_data.iterrows():
    for k, v in row.items():
        if "City" in k:
            if v not in city_names_in_sheet:
                city_names_in_sheet.append(v)

            

In [7]:
city_names_in_sheet

[nan, 'Mumbai', 'Delhi NCR', 'Ahmedabad', 'Bangalore', 'Pune']

In [8]:
college_names_in_sheet = []
for it, row in schol_data.iterrows():
    for k, v in row.items():
        if "College" in k:
            if v not in college_names_in_sheet:
                college_names_in_sheet.append(v)

college_names_in_sheet

['IIT', 'NIT', nan]

In [9]:
schol_data.shape

(990, 38)

In [11]:
schol_data.iloc[92]

Sl No                                                                                 94.0
Scholarship Name                         FINANCIAL SUPPORT TO THE STUDENTS OF NER FOR H...
Status                                                                                 NaN
Class 11 can Apply                                                                      No
Class 12 can Apply                                                                      No
Can UG 1st year apply?                                                                 Yes
Gender                                                                                Both
Family Income (in LPA)                                                                 8.0
Category                                                                           General
State 1                                                                              Assam
State 2                                                                  Arunachal Pradesh

In [12]:
schol_data[:93].to_json("public/data/scholarships/scholarship_data.json", orient="records")

In [13]:
schol_data["Category"].unique()

array([nan, 'PWD', 'Others', 'SC/ST', 'General ', 'General'], dtype=object)

In [46]:
mhcet_data = pd.read_csv("MHT CET 2023 All Cutoffs - Cutoffs.csv")
mhcet_cat = pd.read_csv("MHT CET 2023 All Cutoffs - Categories.csv")

In [47]:
mhcet_cat = mhcet_cat.rename(columns = {"Unnamed: 0": "Cat"})

In [48]:
for col in mhcet_cat.columns:
    print(mhcet_cat[col].unique())

['GOPENS' 'GOPENH' 'GSCS' 'GSTS' 'GVJS' 'GNT1S' 'GNT2S' 'GNT3S' 'GOBCS'
 'LOPENS' 'LSCS' 'LSTS' 'LVJS' 'LNT1S' 'LNT2S' 'LNT3S' 'LOBCS' 'PWDOPENS'
 'DEFOPENS' 'TFWS' 'EWS' 'PWDROBC\nS' 'DEFROBCS' 'Unnamed: 1' 'PWDRSCS'
 'GSCH' 'GVJH' 'GNT1H' 'GOBCH' 'LOPENH' 'LOBCH' 'GOPENO' 'GSCO' 'GNT3O'
 'GOBCO' 'LOPENO' 'LSCH' 'GSTH' 'LNT2H' 'GNT2H' 'GNT3H' 'LNT1H' 'LSTH'
 'GNT2O' 'LSTO' 'LOBCO' 'LNT3H' 'PWDOPENH' 'GSTO' 'GVJO' 'LSCO' 'LNT2O'
 'LVJO' 'LNT3O' 'LVJH' 'LNT1O' 'GNT1O' 'PWDROBC' 'PWDOBCS' 'DEFOBCS'
 'DEFRSCS' 'ORPHAN' 'MI' 'DEFRNT3S' 'DEFRNT2S' 'PWDRNT2S' 'DEFRNT1S'
 'DEFRVJS' 'PWDSCS' 'DEFSCS' 'PWDRNT3S' 'PWDRSCH' 'PWDRNT1S' 'PWDRSTS'
 'PWDOBCH' 'PWDRNT2H' 'PWDSCH' 'PWDRNT1H' 'PWDROBC\nH' 'Unnamed: 2'
 'PWDRSTH' 'PWDRVJH' 'PWDRVJS']
['General' 'Female']
['Open' 'SC' 'ST' 'VJ' 'NT' 'OBC' nan 'EWS' 'Orphan' 'Religious Minority']
['Maharashtra' 'Other']
['No' 'Yes']
['No' 'Yes']
['MHT CET']


In [49]:
# required columns
# Gender, Category, State, PWD, Defense, Closing Rank, Institute, Course

all_dic = []
for it, row in mhcet_data.iterrows():
    for k, v in row.items():
        
        if k in ["College", "Course", "Unnamed: 1", "Unnamed: 2"]:
            continue
        else:
            dic = {}
            dic["Institute"] = row["College"]
            dic["Course"] = row["Course"]
            cat_row = dict(mhcet_cat[mhcet_cat["Cat"] == k].iloc[0])
            
            dic["Category"] = cat_row["Category"]
            if cat_row["Gender"] == "General":
                dic["Gender"] = "Gender-Neutral"
            elif cat_row["Gender"] == "Female":
                dic["Gender"] = "Female-Only"
            dic["Defense"] = cat_row["Defense Ward"]
            dic["PWD"]= cat_row["PWD"]
            dic["State"] = cat_row["State"]
            dic["Category_Key"] = k

            if v == "" or pd.isna(v): continue # no rank available

            v = str(v)

            if "\n" in v:
                v = v.split("\n")[0]
            elif " " in v:
                v = v.split(" ")[0]
            elif "." in v:
                v = v.split(".")[0]

            dic["Closing Rank"] = v

            all_dic.append(dic)
            # break 
    
    # break 
            # dic["Category"] = k

            




In [50]:
final_mhcet_data = pd.DataFrame(all_dic)

In [51]:
final_mhcet_data.to_json("public/data/MHTCET/mhtcet_data.json", orient="records")

In [52]:
for it, row in final_mhcet_data.iterrows():
    # Show Both Maharashtra No No General
    # print(row)
    if row["Category"] == "Open" and row["Gender"] == "Gender-Neutral" and row["State"] == "Maharashtra" and row["PWD"] == "No" and row["Defense"] == "No":
        print(row)
    break

Institute       Government College of Engineering, Amravati
Course                                    Civil Engineering
Category                                               Open
Gender                                       Gender-Neutral
Defense                                                  No
PWD                                                      No
State                                           Maharashtra
Category_Key                                         GOPENS
Closing Rank                                          45820
Name: 0, dtype: object


In [50]:
final_mhcet_data.columns

Index(['Institute', 'Course', 'Category', 'Gender', 'Defense', 'PWD', 'State',
       'Category_Key', 'Closing Rank'],
      dtype='object')

In [53]:
for col in final_mhcet_data.columns:
    if col not in ["Institute", "Closing Rank", "Course"]:
        print(final_mhcet_data[col].unique())

['Open' 'SC' 'ST' 'VJ' 'NT' 'OBC' nan 'EWS' 'Orphan' 'Religious Minority']
['General' 'Female']
['No' 'Yes']
['No' 'Yes']
['Maharashtra']
['GOPENS' 'GSCS' 'GSTS' 'GVJS' 'GNT1S' 'GNT2S' 'GNT3S' 'GOBCS' 'LOPENS'
 'LSCS' 'LSTS' 'LVJS' 'LNT1S' 'LNT2S' 'LNT3S' 'LOBCS' 'PWDOPENS'
 'DEFOPENS' 'TFWS' 'EWS' 'PWDROBC\nS' 'DEFROBCS' 'PWDRSCS' 'GOPENH' 'GSCH'
 'GVJH' 'GNT1H' 'GOBCH' 'LOPENH' 'LOBCH' 'LSCH' 'GNT2H' 'GNT3H' 'LNT1H'
 'GSTH' 'LSTH' 'LNT3H' 'PWDOPENH' 'LNT2H' 'LVJH' 'PWDROBC' 'PWDOBCS'
 'DEFOBCS' 'DEFRSCS' 'DEFRNT3S' 'ORPHAN' 'DEFRNT2S' 'DEFRVJS' 'PWDRNT2S'
 'MI' 'PWDSCS' 'DEFSCS' 'DEFRNT1S' 'PWDRNT1S' 'PWDRSTS' 'PWDRNT3S'
 'PWDOBCH' 'PWDSCH' 'PWDRNT1H' 'PWDROBC\nH' 'PWDRSTH']


In [58]:
final_mhcet_data[final_mhcet_data["Category_Key"] == "LOBCO"]

Empty DataFrame
Columns: [Institute, Course, Category, Gender, Defense, PWD, State, Category_Key, Closing Rank]
Index: []

In [66]:
kcet_data = pd.read_csv("kcet_counselling_data_2022.csv")

print(kcet_data.shape)
print(kcet_data.columns)

(138775, 14)
Index(['ENTRY No.', 'OFFSET', 'Course Type', 'category', 'filename',
       'sheetname', 'college', 'course', 'Caste', 'Category', 'Language',
       'Rural / Urban', 'Label', 'Closing Rank'],
      dtype='object')


In [70]:
all_dic = []
for it, row in kcet_data.iterrows():
    dic = {}
    if pd.isna(row["college"]): continue
    if pd.isna(row["Course Type"]): continue
    dic["Institute"] = row["college"].replace("\xa0", " ")
    dic["Course Type"] = row["Course Type"]
    dic["Course"] = row["course"]
    dic["Category"] = row["Caste"]
    dic["State"] = row["Category"]
    dic["Language"] = row["Language"]
    dic["Rural/Urban"] = row["Rural / Urban"]
    dic["Category_Key"] = row["Label"]

    v = row["Closing Rank"]
    if v == "" or pd.isna(v): continue # no rank available

    v = str(v)

    if "\n" in v:
        v = v.split("\n")[0]
    elif " " in v:
        v = v.split(" ")[0]
    elif "." in v:
        v = v.split(".")[0]

    dic["Closing Rank"] = v

    all_dic.append(dic)

In [71]:
final_kcet_data = pd.DataFrame(all_dic)

In [72]:
for col in final_kcet_data.columns:
    print(col)
    print(final_kcet_data[col].unique())

Institute
['M001  Bangalore Medical College, Bangalore'
 'M003  Kempegowda Institute of Medical Sciences, Bangalore'
 'M004  M.S.Ramaiah Medical College, Bangalore'
 'M005  Khaja Bande Navaz Institute Of Medical Sciences, Gulbarga'
 'M008  M.V.J.Medical College and Research Hospital, Channasandra, Bangalore'
 'M009  Vydehi Institute of Medical Science and Research Centre, Bangalore'
 'M010  A.J.Institute of Medical Sciences, Mangalore'
 'M020  Sri Basaveshwara Medical College and Hospital, Chitradurga'
 'M021  Mysore Medical College, Mysore'
 'M023  Adichunchanagiri Institute of Medical Sciences, Bellur, Mandya Dist.'
 'M024  Mandya Institute of Medical Sciences, Mandya'
 'M025  Hassan Institute of Medical Sciences, Hassan'
 'M031  Karnataka Institute of Medical Sciences, Hubli'
 'M035  S. Nijalingappa Medical College and Research Centre, Bagalkot'
 'M036  SDM College of Medical Sciences and Hospital, Dharwad'
 'M037  Belgaum Institute of Medical Sciences, Belgaum'
 'M042  Mahadevappa 

In [73]:
final_kcet_data.columns

Index(['Institute', 'Course Type', 'Course', 'Category', 'State', 'Language',
       'Rural/Urban', 'Category_Key', 'Closing Rank'],
      dtype='object')

In [74]:
final_kcet_data.to_json("public/data/KCET/kcet_data.json", orient="records")

In [34]:
final_kcet_data[final_kcet_data["Language"] == "Kannada"]["State"].unique()

array(['Karnataka'], dtype=object)

In [54]:
jac_data = pd.read_csv("JAC 2023 Cutoffs - Cutoffs.csv")

jac_data.head()

Institute  \
0  Delhi Technological University (DTU)   
1  Delhi Technological University (DTU)   
2  Delhi Technological University (DTU)   
3  Delhi Technological University (DTU)   
4  Delhi Technological University (DTU)   

                                            Course  EWCWD  EWCWO  EWGLD  \
0           Computer Science and Engineering (CSE)    NaN    NaN    NaN   
1                      Information Technology (IT)    NaN    NaN    NaN   
2                        Software Engineering (SE)    NaN    NaN    NaN   
3                  Mathematics and Computing (MCE)    NaN    NaN    NaN   
4  Electronics and Communication Engineering (ECE)    NaN    NaN    NaN   

   EWGLO  EWGND    EWGNO  EWPDD  EWPDO  ...  SCPDO       SG  STCWD  STCWO  \
0    NaN  24129   7658.0    NaN    NaN  ...    NaN  14322.0    NaN    NaN   
1    NaN  29278   8189.0    NaN    NaN  ...    NaN  22172.0    NaN    NaN   
2    NaN  33772   9246.0    NaN    NaN  ...    NaN  23161.0    NaN    NaN   
3    NaN  37809   9555.0    NaN    NaN  ...    NaN  25761.0    NaN    NaN   
4    NaN  46787  13415.0    NaN    NaN  ...    NaN  33648.0    NaN    NaN   

   STGLD  STGLO     STGND     STGNO  STPDD  STPDO  
0    NaN    NaN  296967.0   96383.0    NaN    NaN  
1    NaN    NaN  412802.0  121909.0    NaN    NaN  
2    NaN    NaN  500239.0  116361.0    NaN    NaN  
3    NaN    NaN  626554.0  143319.0    NaN    NaN  
4    NaN    NaN  917669.0  168274.0    NaN    NaN  

[5 rows x 44 columns]

In [55]:
jac_cat = pd.read_csv("JAC 2023 Cutoffs - Category_Trans.csv")

jac_cat.head()

Category    Caste         Region  Gender Defense Ward Physically Disabled
0    GNCWD  General          Delhi     Any          Yes                  No
1    GNPDD  General          Delhi     Any           No                 Yes
2    GNGLD  General          Delhi  Female           No                  No
3    GNGND  General          Delhi     Any           No                  No
4    GNCWO  General  Outside Delhi     Any          Yes                  No

In [62]:
all_dic = []
for it, row in jac_data.iterrows():
    for k, v in row.items():
        
        if k in ["Institute", "Course"]:
            continue
        else:
            if v == "" or pd.isna(v): continue # no rank available
            
            dic = {}
            dic["Institute"] = row["Institute"]
            dic["Course"] = row["Course"]
            cat_row = dict(jac_cat[jac_cat["Category"] == k].iloc[0])
            
            dic["Category"] = cat_row["Caste"]
            if cat_row["Gender"] == "Any":
                dic["Gender"] = "Gender-Neutral"
            elif cat_row["Gender"] == "Female":
                dic["Gender"] = "Female-Only"
            dic["Defense"] = cat_row["Defense Ward"]
            dic["PWD"]= cat_row["Physically Disabled"]
            dic["State"] = cat_row["Region"]
            dic["Category_Key"] = k

            v = str(v)

            if "\n" in v:
                v = v.split("\n")[0]
            elif " " in v:
                v = v.split(" ")[0]
            elif "." in v:
                v = v.split(".")[0]

            dic["Closing Rank"] = v

            all_dic.append(dic)




In [63]:
jac_final = pd.DataFrame(all_dic)

In [64]:
jac_final.head()

Institute  \
0  Delhi Technological University (DTU)   
1  Delhi Technological University (DTU)   
2  Delhi Technological University (DTU)   
3  Delhi Technological University (DTU)   
4  Delhi Technological University (DTU)   

                                   Course Category          Gender Defense  \
0  Computer Science and Engineering (CSE)      EWS  Gender-Neutral      No   
1  Computer Science and Engineering (CSE)      EWS  Gender-Neutral      No   
2  Computer Science and Engineering (CSE)  General  Gender-Neutral      No   
3  Computer Science and Engineering (CSE)  General  Gender-Neutral      No   
4  Computer Science and Engineering (CSE)      OBC  Gender-Neutral      No   

  PWD          State Category_Key Closing Rank  
0  No          Delhi        EWGND        24129  
1  No  Outside Delhi        EWGNO         7658  
2  No          Delhi        GNGND        12948  
3  No  Outside Delhi        GNGNO         4876  
4  No          Delhi        OBGND        49995

In [10]:
for col in jac_final.columns:
    print(jac_final[col].unique())

['Delhi Technological University (DTU)'
 'Netaji Subhas University of Technology (NSUT)'
 'Indraprastha Institute of Information Technology Delhi (IIIT Delhi)'
 'Indira Gandhi Delhi Technical University for Women (IGDTUW)']
['Computer Science and Engineering (CSE)' 'Information Technology (IT)'
 'Software Engineering (SE)' 'Mathematics and Computing (MCE)'
 'Electronics and Communication Engineering (ECE)'
 'Electrical Engineering (EE)' 'Mechanical Engineering (ME)'
 'Automotive Engineering (MAM)' 'Engineering Physics (EP)'
 'Chemical Engineering (CHE)' 'Civil Engineering (CE)'
 'Production & Industrial Engineering (PIE)'
 'Environmental Engineering (ENE)' 'Bio-Technology (BT)'
 'Computer Science and Engineering (Artificial Intelligence) (CSAI)'
 'Computer Science and Engineering (Data Science) (CSDS)'
 'Information Technology (Network and Information Security) (ITNS)'
 'Mathematics and Computing (MAC)'
 'Electronics and Communication Engineering (Internet of Things) (EIOT)'
 'Instrume

In [65]:
jac_final.to_json("public/data/JEE/jac_data.json", orient="records")